In [1]:
import pandas as pd

# drop unnamed columns 
# drop duplicates
# drop 'Proj. Time' column
# drop any 'Duo' 

# questions
# should I convert to seconds or minutes?
# does the sub division matter? VI, MI, 

In [2]:
df15 = pd.read_csv('data/marathon_results_2015.csv')
df16 = pd.read_csv('data/marathon_results_2016.csv')
df17 = pd.read_csv('data/marathon_results_2017.csv')
df18 = pd.read_csv('data/marathon_results_2018.csv')
print(df18.shape)
df18.head()

(24087, 26)


,Unnamed: 0,Unnamed: 0.1,Bib,Name,Age,M/F,City,State,Country,Citizen,...,25K,30K,35K,40K,Pace,Proj. Time,Official Time,Overall,Gender,Division
0,0,0,189,"Ash, Alan",26,M,Atmore,AL,USA,NaN,...,1:31:16,1:50:19,2:09:22,2:28:08,0:05:58,NaN,2:36:04,63,63,59
1,1,1,855,"Goodman, John Mark",38,M,Birmingham,AL,USA,NaN,...,1:34:33,1:53:49,2:13:27,2:33:06,0:06:11,NaN,2:41:52,187,186,173
2,2,2,1049,"Mcclung, Joshua",34,M,Birmingham,AL,USA,NaN,...,1:35:31,1:55:14,2:15:14,2:34:45,0:06:15,NaN,2:43:42,247,246,226
3,3,3,303,"Graham, James",26,M,Birmingham,AL,USA,NaN,...,1:33:09,1:52:29,2:13:49,2:37:17,0:06:23,NaN,2:47:07,385,377,346
4,4,4,5267,"Pierce, Nathaniel",31,M,Huntsville,AL,USA,NaN,...,1:40:23,2:00:42,2:21:22,2:41:26,0:06:30,NaN,2:50:19,608,593,536


In [3]:
# find duplicates
df18.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24087 entries, 0 to 24086
Data columns (total 26 columns):
Unnamed: 0       24087 non-null int64
Unnamed: 0.1     24087 non-null int64
Bib              24087 non-null object
Name             24087 non-null object
Age              24087 non-null int64
M/F              24087 non-null object
City             24087 non-null object
State            20050 non-null object
Country          24087 non-null object
Citizen          1417 non-null object
Blank            106 non-null object
5K               24087 non-null object
10K              24087 non-null object
15K              24087 non-null object
20K              24087 non-null object
Half             24087 non-null object
25K              24087 non-null object
30K              24087 non-null object
35K              24087 non-null object
40K              24087 non-null object
Pace             24087 non-null object
Proj. Time       245 non-null object
Official Time    24087 non-null object
Ov

In [20]:
# df18['Blank'].value_counts()

# df18[df18['State'] == 'MA']['M/F'].value_counts()
df18['Age'].value_counts()

45    1002
40     910
46     897
47     888
50     791
41     785
48     746
42     706
37     682
43     679
36     678
49     667
51     667
55     646
35     639
38     626
44     621
52     606
39     574
56     538
53     512
32     511
31     505
54     500
27     499
28     489
29     481
60     477
30     476
34     473
      ... 
61     350
24     331
58     293
59     271
62     238
23     225
63     205
65     195
22     153
66     143
64     119
21     111
67     105
68      77
70      67
20      67
71      61
69      44
19      27
72      20
75      14
73      12
18      10
76      10
74       8
80       5
77       3
82       1
81       1
78       1
Name: Age, Length: 64, dtype: int64

In [12]:
df17[df17['State'] == 'MA']['M/F'].value_counts()

F    2467
M    2119
Name: M/F, dtype: int64

In [9]:
print(df17.shape)
df17.head()

(26410, 25)


,Unnamed: 0,Bib,Name,Age,M/F,City,State,Country,Citizen,Unnamed: 9,...,25K,30K,35K,40K,Pace,Proj Time,Official Time,Overall,Gender,Division
0,0,11,"Kirui, Geoffrey",24,M,Keringet,NaN,KEN,NaN,NaN,...,1:16:59,1:33:01,1:48:19,2:02:53,0:04:57,-,2:09:37,1,1,1
1,1,17,"Rupp, Galen",30,M,Portland,OR,USA,NaN,NaN,...,1:16:59,1:33:01,1:48:19,2:03:14,0:04:58,-,2:09:58,2,2,2
2,2,23,"Osako, Suguru",25,M,Machida-City,NaN,JPN,NaN,NaN,...,1:17:00,1:33:01,1:48:31,2:03:38,0:04:59,-,2:10:28,3,3,3
3,3,21,"Biwott, Shadrack",32,M,Mammoth Lakes,CA,USA,NaN,NaN,...,1:17:00,1:33:01,1:48:58,2:04:35,0:05:03,-,2:12:08,4,4,4
4,4,9,"Chebet, Wilson",31,M,Marakwet,NaN,KEN,NaN,NaN,...,1:16:59,1:33:01,1:48:41,2:05:00,0:05:04,-,2:12:35,5,5,5


In [ ]:
df17[df17['State'] == 'MA']

In [ ]:
df17.info()

In [ ]:
df17['Unnamed: 9'].value_counts()

In [ ]:
print(df16.shape)
df16.head()

In [ ]:
print(df15.shape)
df15.head()